In [2]:
import tabula
import os
import pandas as pd
pdf_folder = r"C:\Users\Admin\Downloads\null\New folder"
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]


In [3]:
def extract_and_transform_tables(pdf_file):
    tables = tabula.read_pdf(pdf_file, pages='all', multiple_tables=True)
    transformed_tables = []

    for table in tables:
        if not table.empty:
            dimension_columns = []
            measure_columns = []

            for column in table.columns:
                column_values = table[column].astype(str)  # Convert column values to strings
                if column_values.str.replace(',', '', 1).str.replace('.', '', 1).str.isnumeric().all():
                    measure_columns.append(f"{column}:{','.join(column_values)}")
                else:
                    dimension_columns.append(f"{column}:{','.join(column_values)}")

            transformed_tables.append({
                'Dimension': ', '.join(dimension_columns),
                'Measures': ', '.join(measure_columns)
            })

    return pd.DataFrame(transformed_tables)


In [4]:
result_df = pd.DataFrame()

for pdf_path in pdf_files:
    df = extract_and_transform_tables(pdf_path)
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df.to_csv('assignment2_result.csv', index=False)

In [9]:
#################Second Approach ##############################

In [ ]:
import pdfplumber
import os

def extract_and_transform_tables(pdf_file):
    pdf = pdfplumber.open(pdf_file)
    transformed_tables = []

    for page in pdf.pages:
        tables = page.extract_tables()

        for table in tables:
            dimension_columns = []
            measure_columns = []

            for row in table:
                for cell in row:
                    if cell is not None:
                        if row.index(cell) % 2 == 0:
                            dimension_columns.append(cell)
                        else:
                            measure_columns.append(cell)

            # Join non-empty cells into strings
            dimension_str = ', '.join(cell for cell in dimension_columns if cell is not None)
            measure_str = ', '.join(cell for cell in measure_columns if cell is not None)

            transformed_tables.append({
                'Dimension': dimension_str,
                'Measures': measure_str
            })

    return pd.DataFrame(transformed_tables)

# Process all PDF files in a folder
pdf_folder = 'pdf_null'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

result_df = pd.DataFrame()

for pdf_path in pdf_files:
    df = extract_and_transform_tables(pdf_path)
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df.to_csv('assignment2_result.csv', index=False)
